In [ ]:
import h5py

with h5py.File('rec_autobus.mat', 'r') as f:
    print("Available datasets:", list(f.keys()))
    # grab the Dataset objects
    ds_Fs = f['Fs']
    ds_s  = f['s']
    ds_v  = f['v']
    ds_x  = f['x']

    # now read them into numpy arrays (or a scalar)
    Fs = ds_Fs[()]      # same as ds_Fs[:]
    bruh_s  = ds_s[()]
    bruh_v  = ds_v[()]
    x  = ds_x[()]

    print(f"Fs → {Fs!r}  shape={getattr(Fs, 'shape', 'scalar')}")
    print(f"s  → array of length {bruh_s.shape}")
    print(f"v  → array of length {bruh_v.shape}")
    print(f"x  → array of length {x.shape}")


In [ ]:
import numpy as np

# print the signal to noise ratio in time mean over each frame
# s is signal vector, v is noise vector

bruh_s = bruh_s.flatten()
bruh_v = bruh_v.flatten()

window = 1024
overlap = 512
hop = window - overlap
snrs = []

for i in range(0, len(bruh_s), hop):
    # get the current window
    s_window = bruh_s[i:i + window]
    v_window = bruh_v[i:i + window]

    # calculate the mean of the signal and noise
    s_mean = np.mean(s_window**2)
    v_mean = np.mean(v_window**2)

    # calculate the SNR
    snr = 10 * np.log10(s_mean / v_mean)
    snrs.append(snr)

    print(f"SNR for window {i//hop}: {snr:.2f} dB")


# plot the SNR
import matplotlib.pyplot as plt

# create a time vector
snrs = np.array(snrs)
time = np.arange(0, len(snrs))

#time = range(0, len(snrs))

print(snrs.shape)
print(time.shape)
plt.plot(time.flatten(), snrs.flatten())
plt.xlabel('Time (s)')
plt.ylabel('SNR (dB)')
plt.title('Signal to Noise Ratio')
plt.grid()
plt.show()


In [ ]:
import h5py

with h5py.File('speech_bus_two_mics.mat', 'r') as f:
    print("Available datasets:", list(f.keys()))
    # grab the Dataset objects
    ds_fs    = f['fs']
    ds_s1    = f['s_mic1']
    ds_s2    = f['s_mic2']
    ds_v1    = f['v_mic1']
    ds_v2    = f['v_mic2']
    ds_x1    = f['x_mic1']
    ds_x2    = f['x_mic2']

    # now read them into numpy arrays (or a scalar)
    fs      = ds_fs[()]     # sample rate (scalar)
    s_mic1  = ds_s1[()]     # time-series mic1
    s_mic2  = ds_s2[()]     # time-series mic2
    v_mic1  = ds_v1[()]     # voltage mic1
    v_mic2  = ds_v2[()]     # voltage mic2
    x_mic1  = ds_x1[()]     # whatever x_mic1 is
    x_mic2  = ds_x2[()]     # whatever x_mic2 is

    print(f"fs      → {fs!r}   (scalar)")
    print(f"s_mic1  → array, shape {s_mic1.shape}")
    print(f"s_mic2  → array, shape {s_mic2.shape}")
    print(f"v_mic1  → array, shape {v_mic1.shape}")
    print(f"v_mic2  → array, shape {v_mic2.shape}")
    print(f"x_mic1  → array, shape {x_mic1.shape}")
    print(f"x_mic2  → array, shape {x_mic2.shape}")


In [ ]:
# stft of the signal
#S_mic1 = np.fft.fft(s_mic1)
#V_mic1 = np.fft.fft(v_mic1)
#X_mic1 = np.fft.fft(x_mic1)

import numpy as np
from scipy.signal import stft

# assume you already have s_mic1, v_mic1, x_mic1 and your sampling rate fs

# choose your STFT parameters:
nperseg  = 1024   # window length in samples
noverlap = nperseg // 2 # overlap in samples
window   = 'hann'

# STFT of mic1 signal
f_s, t_s, Sxx_mic1 = stft(s_mic1, fs=fs, window=window,
                          nperseg=nperseg, noverlap=noverlap,
                          boundary=None)

# STFT of the noise channel
f_v, t_v, Sxx_vmic1 = stft(v_mic1, fs=fs, window=window,
                           nperseg=nperseg, noverlap=noverlap,
                           boundary=None)

# STFT of your x channel
f_x, t_x, Sxx_xmic1 = stft(x_mic1, fs=fs, window=window,
                           nperseg=nperseg, noverlap=noverlap,
                           boundary=None)


# wiener filter
# w(k,l) = abs(Sxx_mic1(k,l))^2 / (abs(Sxx_mic1(k,l))^2 + abs(Sxx_vmic1(k,l))^2)

W_mic1 = np.abs(Sxx_mic1)**2 / (np.abs(Sxx_mic1)**2 + np.abs(Sxx_vmic1)**2)

print(W_mic1.shape)
# print variance of W_mic1
print(np.var(W_mic1))

# apply the wiener filter to the x channel
Sxx_xmic1_wiener = Sxx_xmic1 * W_mic1
# inverse STFT to get the filtered signal
from scipy.signal import istft
_, filtered_signal = istft(Sxx_xmic1_wiener, fs=fs, window=window,
                            nperseg=nperseg, noverlap=noverlap,
                            boundary=None)

# play the filtered signal
import sounddevice as sd
sd.play(filtered_signal, samplerate=fs)
sd.wait()  # Wait until sound has finished playing



In [ ]:
import soundfile as sf
import IPython.display as ipd

# STFT of the noise channel
f_v2, t_v2, Sxx_vmic2 = stft(v_mic2, fs=fs, window=window,
                           nperseg=nperseg, noverlap=noverlap,
                           boundary=None)

# wiener filter = max(0, abs(X(k,l))**2 ) - abs(Sxx_vmic2(k,l))**2 / (abs(X(k,l))**2 + np.eps)

W_mic2 = np.maximum(0, np.abs(Sxx_xmic1)**2 - np.abs(Sxx_vmic2)**2) / (np.abs(Sxx_xmic1)**2 + np.finfo(float).eps)

print(W_mic2.shape)
# print variance of W_mic2
print(np.var(W_mic2))

# apply the wiener filter to the x channel
Sxx_xmic1_2_wiener = Sxx_xmic1 * W_mic2

# play the sound
_, filtered_signal_2 = istft(Sxx_xmic1_2_wiener, fs=fs, window=window,
                            nperseg=nperseg, noverlap=noverlap,
                            boundary=True)

sf.write("susbus.wav", filtered_signal_2, fs)
ipd.Audio("susbus.wav")

In [ ]:
# compute the SNR for filtered_signal_2 to  v_mic2 and filtered_signal to v_mic1

import numpy as np
import matplotlib.pyplot as plt

# assume these are already defined and 1D:
# filtered_signal, v_mic1, filtered_signal_2, v_mic2, and your sample rate fs if you want time axis

pairs = [
    ("Mic1", filtered_signal, v_mic1),
    ("Mic2", filtered_signal_2, v_mic2),
]

window  = 1024
overlap = 0
hop      = window - overlap

plt.figure(figsize=(10, 4 * len(pairs)))

snrss = []

for idx, (label, bruh_s, bruh_v) in enumerate(pairs, 1):
    bruh_s = bruh_s.flatten()
    bruh_v = bruh_v.flatten()
    snrs = []

    # frame‐wise SNR
    for i in range(0, min(len(bruh_s), len(bruh_v)) - window + 1, hop):
        s_win = bruh_s[i:i + window]
        v_win = bruh_v[i:i + window]
        # mean squared
        s_mean = np.mean(np.abs(s_win)**2)
        v_mean = np.mean(np.abs(v_win)**2)
        snr     = 10 * np.log10(s_mean / v_mean)
        snrs.append(snr)

    snrs = np.array(snrs)
    time = np.arange(len(snrs)) * hop / fs  # in seconds
    time = time.squeeze()

    print(f"=== {label} ===")
    for frame_idx, val in enumerate(snrs):
        print(f"{label} SNR frame {frame_idx}: {val:.2f} dB")


    snrss.append(snrs)
    # plot
    ax = plt.subplot(len(pairs), 1, idx)
    ax.plot(time, snrs)
    ax.set_ylabel("SNR (dB)")
    ax.set_xlabel("Time (s)")
    ax.set_title(f"Frame‐wise SNR for {label}")
    ax.grid(True)

plt.tight_layout()
plt.show()

print(len(snrss[0]))
print(len(snrss[1]))

# print the average SNR for each mic
print(f" average SNR for exp1: {np.mean(snrss[0])} dB")
print(f" average SNR for exp2: {np.mean(snrss[1])} dB")